# This code is only to demonstrate the process of high-definition with CodeFormer after wav2lip reasoning.



In [7]:
#@title Download wav2lip source code and weights and install wav2lip environment

!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa
!pip install basicsr

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output 
clear_output()
print("\nDone")


Done


In [3]:
#@title Download CodeFormer's source code weights and install dependencies

# Download the source code of CodeFormer
%cd /content
!git clone https://github.com/sczhou/CodeFormer.git
%cd /content/CodeFormer

/content
Cloning into 'CodeFormer'...
remote: Enumerating objects: 583, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 583 (delta 168), reused 166 (delta 144), pack-reused 349
Receiving objects: 100% (583/583), 17.31 MiB | 9.02 MiB/s, done.
Resolving deltas: 100% (279/279), done.
/content/CodeFormer


In [4]:
# Download CodeFormer Weights
%cd /content/CodeFormer
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

/content/CodeFormer
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/CodeFormer/weights/facelib/detection_Resnet50_Final.pth

100% 104M/104M [00:00<00:00, 361MB/s] 
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/parsing_parsenet.pth" to /content/CodeFormer/weights/facelib/parsing_parsenet.pth

100% 81.4M/81.4M [00:00<00:00, 330MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.1

In [5]:
# Install CodeFormer dependencies
%cd /content/CodeFormer
!python basicsr/setup.py develop

/content/CodeFormer
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issue

In [6]:
#@title Upload video and audio
# Create related folders
%cd /content
!mkdir output 
!mkdir input 
# Upload audio and video
%cd /content/input 
from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content
/content/input


Saving kimk_audio.mp3 to kimk_audio.mp3
Saving kimk_7s_raw.mp4 to kimk_7s_raw.mp4
Saving kim_7s_raw.mp4 to kim_7s_raw.mp4
Saving kim_audio.mp3 to kim_audio.mp3


In [7]:
#@title Use wav2lip to initially generate a modified mouth video

%cd /content

!mkdir output 
!mkdir images 

%cd /content/CodeFormer

!mkdir results 

/content
mkdir: cannot create directory ‘output’: File exists
/content/CodeFormer


In [1]:
inputAudioPath = '/content/input/kim_audio.mp3'
inputVideoPath = '/content/input/kimk_7s_raw.mp4'

In [2]:
# Inference with wav2lip
# checkpoints can use wav2lip.pth or wav2lip_gan.pt
%cd /content/Wav2Lip 
!python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face {inputVideoPath} --audio {inputAudioPath}

/content/Wav2Lip
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 253
Extracting raw audio...
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsn

In [3]:
# Convert the video generated by Wav2Lip into a picture and save it to images
%cd /content
!mkdir images
%cd /content/Wav2Lip
import cv2
from tqdm import tqdm
from os import path
import os

vidcap = cv2.VideoCapture("/content/Wav2Lip/results/result_voice.mp4")
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join('/content/images', str(frameNumber).zfill(4)+'.jpg'), image)

/content
mkdir: cannot create directory ‘images’: File exists
/content/Wav2Lip
FPS:  30.0 Frames:  218


100%|██████████| 218/218 [00:11<00:00, 19.72it/s]


In [4]:
#@title Face Restoration with CodeFormer

%cd /content/CodeFormer
# Define related functions
import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

/content/CodeFormer


In [5]:
# CodeFormer for inference
%cd /content/CodeFormer
## Only fix the face
# CODEFORMER_FIDELITY = 0.5  
# !python inference_codeformer.py -w $CODEFORMER_FIDELITY --has_aligned --input_path /content/images

# whole images ##Overall picture repair
CODEFORMER_FIDELITY = 0.7
!python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path /content/images --bg_upsampler realesrgan

/content/CodeFormer
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/RealESRGAN_x2plus.pth" to /content/CodeFormer/weights/realesrgan/RealESRGAN_x2plus.pth

100% 64.0M/64.0M [00:03<00:00, 19.0MB/s]
Face detection model: retinaface_resnet50
Background upsampling: True, Face upsampling: False
[1/218] Processing: 0000.jpg
	detect 4 faces
[2/218] Processing: 0001.jpg
	detect 4 faces
[3/218] Processing: 0002.jpg
	detect 4 faces
[4/218] Processing: 0003.jpg
	detect 4 faces
[5/218] Processing: 0004.jpg
	detect 4 faces
[6/218] Processing: 0005.jpg
	detect 4 faces
[7/218] Processing: 0006.jpg
	detect 4 faces
[8/218] Processing: 0007.jpg
	detect 4 faces
[9/218] Processing: 0008.jpg
	detect 4 faces
[10/218] Processing: 0009.jpg
	detect 4 faces
[11/218] Processing: 0010.jpg
	detect 4 faces
[12/218] Processing: 0011.jpg
	detect 4 faces
[13/218] Processing: 0012.jpg
	detect 4 faces
[14/218] Processing: 0013.jpg
	detect 4 faces
[15/218] Processing: 0014.jpg
	detect 4 faces


In [10]:
# Generate a new repaired video
import os
restoredFramesPath = '/content/CodeFormer/results/images_0.7' + '/final_results/'
processedVideoOutputPath = '/content/output'

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

batch = 0
batchSize = 600
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end)
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)


  out = cv2.VideoWriter(processedVideoOutputPath+'/batch_'+str(batch).zfill(4)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
  batch = batch + 1
 
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

#audio and video merge Generate audio video
audio = ffmpeg.input(f'{inputAudioPath}')
video = ffmpeg.input(f'/content/output/batch_0000.avi')
print("Merge video and audio")
out = ffmpeg.output(video, audio, f'/content/output/final_output.mp4')
out.run()  
print("Congratulations, the audio and video merge is completed and stored in output/final_output.mp4")

finalProcessedOuputVideo = "/content/output/final_output.mp4"
from google.colab import files
files.download(finalProcessedOuputVideo)

  0%|          | 0/1 [00:00<?, ?it/s]

processing  0 600



100%|██████████| 1/1 [00:54<00:00, 54.82s/it]


Merge video and audio
Congratulations, the audio and video merge is completed and stored in output/final_output.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>